In [1]:
import sys
import random
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import nt_search
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [8]:
Data = pd.read_csv("/pod/2/ke-lab/LUOZ/m6AAI/Figure2/Fasta_SNM_LongInternalExon.csv", keep_default_na=False)

In [9]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,...,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength,POS,Up,Mid,Down
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,...,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071,74096,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,CAGTACCGACTCCAGCCACTGCAAGGGTCAGGAGTCAAGACTCAGA...,CTGCTTCCTTCCAGCTGCCTCTGGAATCAAGGCCGAGCATATCGCT...
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,...,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551,448476,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,CAGTGGAAAATACTGCCGGCTATGCGATATTCAGTTCAATAATCTT...,TCAAATGAACTAACTGAGTTACTAAAGAAAGCAGTCACCTTTGGTA...
2,Ppp1r15b,chr1,+,133131142,133139783,133131746,133136887,133136662,133139783,3121,...,133136855,133136856,0.909690,1,GAGTAAGGAAGCCGACAGGGACCACAGACGCGGCCAATCAGAGGCC...,8641,5713,GAGTAAGGAAGCCGACAGGGACCACAGACGCGGCCAATCAGAGGCC...,CTTTGAGCACAGAGAAAAAATGTTTAATAGACTGAGGATCGAGTCA...,CACTCTGTCTCTTACTTGAGAGTTTCCCTTAAAAACAAACACTGGC...
3,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,...,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745,3268,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,ATATCCTGTGATTTTGAAGAGGCGGCAGAGCCCGTTCTCATCCAAG...,AATGGCCAGCTCCATTTGGTGGGTTATTCTGACACTCACTTGGTTT...
4,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,...,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200,18163,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,GGTGCCTGTGCCCGGCAGCCTGGACACAGAGCTGGAAAGCCGGGCT...,GGTTGTTTTCTGTTTGAGGGTGTAAGGAACAATGTTGGTGTGCACA...
5,Gpatch2,chr1,+,187215510,187351427,187215623,187348577,187225502,187226219,717,...,187225750,187225751,0.902874,1,TTCCGGAAGTTGCTCTCAAAATGCTGAAGTCCTGTGTGGAATTGGC...,135917,10240,TTCCGGAAGTTGCTCTCAAAATGCTGAAGTCCTGTGTGGAATTGGC...,CCACTGCTTAAGTGAAGGCTCTGATTCTAGTTTAGAAGAACCAAGT...,ACCAGATGTTAGTGGCGAAGCGGAGGCCATCTTCAAACCTAAGCAG...
6,Rif1,chr2,+,52072836,52122381,52073149,52121203,52109910,52113000,3090,...,52112643,52112644,0.898967,1,ATCTTGGTCGTGGAGGAGCGGGCTGCACGCGTGAGTAAATAAGCGC...,49545,39807,ATCTTGGTCGTGGAGGAGCGGGCTGCACGCGTGAGTAAATAAGCGC...,TCAACACCCAGAGCTTTGTTAGTGACATTGAAATGAGTTCTGGAGA...,GCAAAATTAAGTGGCAATGAAGCTACAGTGGGAAATGATACTCTGC...
7,Nufip2,chr11,+,77686154,77712741,77686227,77709735,77691541,77693254,1713,...,77692815,77692816,0.898588,1,CGCGGAGAAGCAGTCTCAGATCCTGACGGTGCAGCAGCCCGCAGCC...,26587,6661,CGCGGAGAAGCAGTCTCAGATCCTGACGGTGCAGCAGCCCGCAGCC...,AGAATCAATGGGGTTTATCATTTATAAATGAGCCCAGTGCTGGCCC...,GGAGAATATCCTGCCACTTTGGTTTCACAGGGTGCTGAAATAATTC...
8,Plk4,chr3,+,40800018,40816883,40800253,40816417,40805007,40805920,913,...,40805493,40805494,0.897574,1,GTCACCGCCAGGCTGGCTCGTTCGGAGACCGGCGGGAATTTTTCAA...,16865,5475,GTCACCGCCAGGCTGGCTCGTTCGGAGACCGGCGGGAATTTTTCAA...,TGTGTTGGACCATCCTTTCATGTCACGAAATCCTTCACCAAAGAGT...,CAACAATTACAGCCTCTTCTGGTACCAGTTTGAGTGGCAGCCTACT...
9,Fbxo30,chr10,+,11281329,11298052,11289535,11295420,11289519,11291572,2053,...,11290915,11290916,0.892698,1,GGTTGGAGGCCGAGGAGAAAAGGCGGCGGTCCGGGCAGGGCGGAGG...,16723,9586,GGTTGGAGGCCGAGGAGAAAAGGCGGCGGTCCGGGCAGGGCGGAGG...,TATCTGATAGCCGCATGACTGATGTTTATCACGTTGACTTTGGGAC...,ATAGCTTCGGCTTCAGCTTGTGATCATGCCAATCCACAGCTTTCAA...


In [10]:
Up = Data["Up"].tolist()
Mid = Data["Mid"].tolist()
Down = Data["Down"].tolist()

### Saturation Single Nucleotide Mutation

In [11]:
def SingleNucleotideMutation(DNA):
    
    DNA = DNA.upper()
    
    if DNA == "A":
        Mutation = ["T","C","G"]
    if DNA == "T":
        Mutation = ["A","C","G"]
    if DNA == "C":
        Mutation = ["T","A","G"]
    if DNA == "G":
        Mutation = ["T","C","A"]
    return Mutation


In [12]:
from itertools import product
comb = list(product(["A","T","C","G"],repeat=5))
FiverMer = []
for i in range(len(comb)):
    value = "".join(comb[i])
    FiverMer.append(value)

In [13]:
Dict = dict(zip(FiverMer, [0]*1024))
for key,value in Dict.items():
    motif = key
    Dict[key] = {-1:[], 0:[], 1:[]}

#### Downstream sequence

In [14]:
for i in range(len(Data)):
    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()

    for j in range(53,97,1):
        DNA = mid[j:(j+1)]
        origin = mid[0:j] + DNA + mid[(j+1):101]
        ori0 = origin[(j-4):(j+1)]
        ori1 = origin[(j-3):(j+2)]
        ori2 = origin[(j-2):(j+3)]
        ori3 = origin[(j-1):(j+4)]
        ori4 = origin[(j-0):(j+5)]
        
        Mutation = SingleNucleotideMutation(DNA)
        
        # Mutation[0]
        seq = mid[0:j] + Mutation[0] + mid[(j+1):101]        
        input_sequence = up + seq + down
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value0 = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue0 = value0 - Data.loc[i,"Probabilty"]
        
        motif0 = seq[(j-4):(j+1)]
        motif1 = seq[(j-3):(j+2)]
        motif2 = seq[(j-2):(j+3)]
        motif3 = seq[(j-1):(j+4)]
        motif4 = seq[(j-0):(j+5)]
        
        List1 = [ori0, ori1, ori2, ori3, ori4]
        List2 = [motif0, motif1, motif2, motif3, motif4]
        List3 = list(set(List1).intersection(set(List2))) # intersect
        List4 = list(set(List1).union(set(List2))) # union
        List5 = list(set(FiverMer).difference(set(List4))) # difference
        
        List1 = list(set(List1).difference(set(List3)))
        List2 = list(set(List2).difference(set(List3)))
                
        for k in range(len(List1)):
            Dict[List1[k]][-1].append(Dvalue0)
        for k in range(len(List2)):
            Dict[List2[k]][1].append(Dvalue0)
        for k in range(len(List5)):
            Dict[List5[k]][0].append(Dvalue0)
        for k in range(len(List3)):
            Dict[List3[k]][0].append(Dvalue0)

        # Mutation[1]
        seq = mid[0:j] + Mutation[1] + mid[(j+1):101]        
        input_sequence = up + seq + down
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value1 = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue1 = value1 - Data.loc[i,"Probabilty"]

        motif0 = seq[(j-4):(j+1)]
        motif1 = seq[(j-3):(j+2)]
        motif2 = seq[(j-2):(j+3)]
        motif3 = seq[(j-1):(j+4)]
        motif4 = seq[(j-0):(j+5)]
        
        List1 = [ori0, ori1, ori2, ori3, ori4]
        List2 = [motif0, motif1, motif2, motif3, motif4]
        List3 = list(set(List1).intersection(set(List2))) # intersect
        List4 = list(set(List1).union(set(List2))) # union
        List5 = list(set(FiverMer).difference(set(List4))) # difference
        
        List1 = list(set(List1).difference(set(List3)))
        List2 = list(set(List2).difference(set(List3)))
        
        
        for k in range(len(List1)):
            Dict[List1[k]][-1].append(Dvalue1)
        for k in range(len(List2)):
            Dict[List2[k]][1].append(Dvalue1)
        for k in range(len(List5)):
            Dict[List5[k]][0].append(Dvalue1)
        for k in range(len(List3)):
            Dict[List3[k]][0].append(Dvalue1)
                     
        
        
        # Mutation[2]
        seq = mid[0:j] + Mutation[2] + mid[(j+1):101]        
        input_sequence = up + seq + down
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value2 = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue2 = value2 - Data.loc[i,"Probabilty"]
        
        motif0 = seq[(j-4):(j+1)]
        motif1 = seq[(j-3):(j+2)]
        motif2 = seq[(j-2):(j+3)]
        motif3 = seq[(j-1):(j+4)]
        motif4 = seq[(j-0):(j+5)]
        
        List1 = [ori0, ori1, ori2, ori3, ori4]
        List2 = [motif0, motif1, motif2, motif3, motif4]
        List3 = list(set(List1).intersection(set(List2))) # intersect
        List4 = list(set(List1).union(set(List2))) # union
        List5 = list(set(FiverMer).difference(set(List4))) # difference
        
        List1 = list(set(List1).difference(set(List3)))
        List2 = list(set(List2).difference(set(List3)))
        
        
        for k in range(len(List1)):
            Dict[List1[k]][-1].append(Dvalue2)
        for k in range(len(List2)):
            Dict[List2[k]][1].append(Dvalue2)
        for k in range(len(List5)):
            Dict[List5[k]][0].append(Dvalue2)
        for k in range(len(List3)):
            Dict[List3[k]][0].append(Dvalue2)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [15]:
for key,value in Dict.items():
    motif = key
    List1 = Dict[motif][-1]
    List2 = Dict[motif][0]
    List3 = Dict[motif][1]
    Score = [-1]*len(List1) + [0]*len(List2) + [1]*len(List3)
    Value = List1 + List2 + List3
    df = pd.DataFrame({"Score":Score, "Value":Value})
    df.to_csv("./Figure2/LongInternalExon/{}.csv".format(motif), index=0)